# Test Lightning Refactor

In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

# Imports
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import optim
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
import yaml
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')
sys.path.append('../scripts')

# Imports
from models.submodels.interaction_gnn import InteractionGNN
from models.submodels.euclidnet import EuclidNet_SO3, EuclidNet_SO2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Load config file
with open('debug_config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
model = InteractionGNN(config)

In [4]:
print(model.hparams)

"aggregation":       mean_sum
"artifacts":         /global/cfs/cdirs/m3443/data/lightning_models/lightning_checkpoints
"batch_size":        4
"batchnorm":         False
"c_weight":          0.1
"data_split":        [100, 100, 10]
"edge_cut":          0.5
"equi_output":       True
"factor":            0.8
"gpus":              1
"hidden_activation": SiLU
"input_dir":         /global/cfs/cdirs/m3443/usr/ameyat/hitgraphs/geometric/1p5
"layernorm":         True
"lr":                0.001
"max_epochs":        3
"model":             InteractionGNN
"n_graph_iters":     3
"n_hidden":          32
"n_input":           3
"n_layers":          3
"n_output":          1
"nodes":             1
"patience":          30
"project":           EuclideanTracking_IN
"pt":                1.5
"truth_key":         y
"vector_dim":        3
"warmup":            10
"weight":            0.5


In [4]:
checkpoint_callback = ModelCheckpoint(
    monitor="eff", mode="max", save_top_k=2, save_last=True
)

logger = WandbLogger(
    project=config["project"], group="InitialTest", save_dir=config["artifacts"]
)

trainer = Trainer(
    gpus=1,
    max_epochs=config["max_epochs"],
    logger=logger,
    callbacks=[checkpoint_callback],
)
trainer.fit(model)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Setting up dataset


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: murnanedaniel. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type       | Params
------------------------------------------------------
0 | node_encoder           | Sequential | 2.2 K 
1 | edge_encoder           | Sequential | 4.2 K 
2 | edge_network           | Sequential | 5.2 K 
3 | node_network           | Sequential | 5.2 K 
4 | output_edge_classifier | Sequential | 5.2 K 
------------------------------------------------------
22.1 K    Trainable params
0         Non-trainable params
22.1 K    Total params
0.088     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_SUPPORTED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`